In [82]:
from sklearn.model_selection import train_test_split
# from sklearn.manifold import LocallyLinearEmbedding
from sklearn.datasets import load_iris, load_digits
import matplotlib.pyplot as plt
import numpy as np
from numpy.typing import NDArray
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.naive_bayes import GaussianNB


In [83]:
data = load_iris()
x = data.data
y = data.target
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [84]:
# # Create and fit LLE model
# lle = LocallyLinearEmbedding(n_neighbors=20, n_components=2)
# X_transformed = lle.fit_transform(x)
# print(X_transformed.shape)

# # Visualize the results
# plt.scatter(X_transformed[:, 0], X_transformed[:, 1], c=y)
# plt.title('LLE Projection of Iris Dataset')
# plt.xlabel('Component 1')
# plt.ylabel('Component 2')
# plt.show()

<div class="markdown prose w-full break-words dark:prose-invert dark"><p>The "Singular matrix" error typically occurs when you try to compute the inverse of a singular or nearly singular matrix. In the context of Locally Linear Embedding (LLE) or similar algorithms, it often means that the local neighborhood for a particular point is not diverse enough, leading to a poorly conditioned matrix.</p><p>To address this issue, you can add a small regularization term to the diagonal of the matrix <code>G</code> before computing the inverse. This regularization helps stabilize the inversion process and avoid singularities. The modified line would look like this:</p><pre><div class="bg-black rounded-md"><div class="flex items-center relative text-gray-200 bg-gray-800 dark:bg-token-surface-primary px-4 py-2 text-xs font-sans justify-between rounded-t-md"></div><div class="p-4 overflow-y-auto"><code class="!whitespace-pre hljs language-python">G_inv = np.linalg.inv(G + <span class="hljs-number">1e-4</span> * np.eye(G.shape[<span class="hljs-number">0</span>]))
</code></div></div></pre><p>Here, <code>1e-4</code> is a small value that you can adjust based on your specific dataset and requirements.</p><p>Feel free to experiment with different values for the regularization term (e.g., <code>1e-4</code>) to find a suitable value for your specific dataset. Adjusting this value can help balance the trade-off between stability and the influence of the regularization term.</p></div>

In [85]:
class LocallyLinearEmbedding:
    n_component: int
    n_neighbours: int
    data: np.ndarray
    weghits: np.ndarray
    eigenvectors: np.ndarray  # Store the eigenvectors from fit_transform
    
    def __init__(self, n_component, n_neighbours) -> None:
        self.n_component = n_component
        self.n_neighbours = n_neighbours
        
    def fit_transform(self, data):
        self.data = data
        self.LinearReconustruction()
        return self.LinearEmbedding()
    
    def LinearEmbedding(self):
        w_size = self.weghits.shape[0]
        M = np.eye(w_size) - self.weghits
        M = M.T @ M
        eigenvalues, eigenvectors = np.linalg.eig(M)
        indices = np.argsort(eigenvalues)[1:self.n_component+1]
        self.eigenvectors = np.real(eigenvectors[:, indices])  
        Y = self.eigenvectors
        return Y
    
    def LinearReconustruction(self):
        w = []  # weights matrix with n*n_neighbours size
        ones = np.ones(self.n_neighbours).reshape(-1, 1)
        for i in range(self.data.shape[0]):
            d = self.data[i].reshape(-1, 1)
            knn, knn_i = self.KNN(i)
            G = d @ ones.T - knn.T
            G = G.T @ G
            G_inv = np.linalg.inv(G + 1e-4 * np.eye(G.shape[0]))
            w_i = ((G_inv @ ones) / (ones.T @ G_inv @ ones)).squeeze()
            w_i = self.ReconstructWeights(w_i, knn_i)
            w.append(w_i)
        self.weghits = np.array(w)
        return w
    
    def ReconstructWeights(self, w, knn_i):
        res = np.zeros((self.data.shape[0]))
        for i in range(res.shape[0]):
            for index in range(len(knn_i)):
                if knn_i[index] == i:
                    res[i] = w[index]
                    break
        return res
    
    def KNN(self, index):
        neighbors_t = {}
        for i in range(self.data.shape[0]):
            if not i == index:
                neighbors_t[i] = self.distance(self.data[i], self.data[index])
        neighbors_t = dict(sorted(neighbors_t.items(), key=lambda item: item[1]))
        neighbors = []
        neighbors_i = []
        for index, key in enumerate(neighbors_t):
            neighbors.append(self.data[key])  # data of key
            neighbors_i.append(key)  # key itself
            if index + 1 == self.n_neighbours:
                break
        return np.array(neighbors), np.array(neighbors_i)
    
    def transform(self, new_data):
        # Step 1: Find the KNN of the new data in the original data set
        w = []
        ones = np.ones(self.n_neighbours).reshape(-1, 1)
        for d in new_data:
            d = d.reshape(-1, 1)
            knn, knn_i = self.KNN_new_point(d)
            G = d @ ones.T - knn.T
            G = G.T @ G
            G_inv = np.linalg.inv(G + 1e-4 * np.eye(G.shape[0]))
            w_i = ((G_inv @ ones) / (ones.T @ G_inv @ ones)).squeeze()
            w_i = self.ReconstructWeights(w_i, knn_i)
            w.append(w_i)
        w_new = np.array(w)
        
        # Step 2: Use the eigenvectors from fit_transform to project the new data
        Y_new = w_new @ self.eigenvectors
        return Y_new
    
    def KNN_new_point(self, point):
        neighbors_t = {}
        for i in range(self.data.shape[0]):
            neighbors_t[i] = self.distance(self.data[i], point.squeeze())
        neighbors_t = dict(sorted(neighbors_t.items(), key=lambda item: item[1]))
        neighbors = []
        neighbors_i = []
        for index, key in enumerate(neighbors_t):
            neighbors.append(self.data[key])  # data of key
            neighbors_i.append(key)  # key itself
            if index + 1 == self.n_neighbours:
                break
        return np.array(neighbors), np.array(neighbors_i)
    
    def distance(self, data1, data2, sigma=0.001):
        # return np.linalg.norm(data1 - data2)
        return np.exp(-np.linalg.norm(data1 - data2)**2 / (2 * sigma**2))
    
    

In [86]:
# LLE = LocallyLinearEmbedding(2, 3)
# x_LLE = [4, 5, 10, 4, 3, 11, 14 , 8, 10, 12]
# y_LLE = [21, 19, 24, 17, 16, 25, 24, 22, 21, 21]
# classes = [0, 0, 1, 0, 0, 1, 1, 0, 1, 1]
# selected = 7
# LLE.data = np.column_stack((x_LLE,y_LLE))
# print(LLE.data[selected])
# n, _=LLE.KNN(selected)
# for i in n:
#     print(i)
#     plt.scatter(i[0], i[1], c='green', alpha=.8, zorder=100, label='neighbours')
# plt.scatter(LLE.data[selected][0], LLE.data[selected][1], c='red', zorder=100, label='selected')
# plt.scatter(x_LLE, y_LLE, c=classes, label='all points')
# plt.legend()
# plt.show()

In [87]:
lle = LocallyLinearEmbedding(n_component=3, n_neighbours=5)
x_new_train = lle.fit_transform(x_train)
x_new_test = lle.transform(x_test)


gnb = GaussianNB()
gnb.fit(x_new_train, y_train)

y_pred_train = gnb.predict(x_new_train)
y_pred_test = gnb.predict(x_new_test)
print(accuracy_score(y_train, y_pred_train))
print(accuracy_score(y_test, y_pred_test))

0.95
0.9666666666666667


In [88]:
from sklearn.manifold import trustworthiness

def calculate_trustworthiness(X_high, X_low, n_neighbors=5):
    score = trustworthiness(X_high, X_low, n_neighbors=n_neighbors)
    return score

# Example usage:
trustworthiness_score = calculate_trustworthiness(x_train, x_new_train)
print(f"Trustworthiness: {trustworthiness_score}")


Trustworthiness: 0.9506696428571428


In [89]:
import numpy as np
from sklearn.metrics import pairwise_distances

def residual_variance(X_high, X_low):
    # Calculate pairwise distances in high-dimensional and low-dimensional spaces
    D_high = pairwise_distances(X_high)
    D_low = pairwise_distances(X_low)

    # Flatten the distance matrices
    D_high_flat = D_high.flatten()
    D_low_flat = D_low.flatten()

    # Calculate the residual variance
    variance_high = np.var(D_high_flat)
    variance_diff = np.var(D_high_flat - D_low_flat)
    residual_variance = variance_diff / variance_high

    return residual_variance

# Example usage:
# X_high is the original high-dimensional data
# X_low is the LLE-embedded data
residual_variance_score = residual_variance(x_train, x_new_train)
print(f"Residual Variance: {residual_variance_score}")


Residual Variance: 0.9297717828846278


In [90]:
from sklearn.neighbors import NearestNeighbors

def calculate_continuity(X_high, X_low, n_neighbors=5):
    nn_high = NearestNeighbors(n_neighbors=n_neighbors).fit(X_high)
    nn_low = NearestNeighbors(n_neighbors=n_neighbors).fit(X_low)

    high_neighbors = nn_high.kneighbors(return_distance=False)
    low_neighbors = nn_low.kneighbors(return_distance=False)

    overlap = 0
    for i in range(X_high.shape[0]):
        overlap += len(np.intersect1d(high_neighbors[i], low_neighbors[i]))

    continuity = overlap / (X_high.shape[0] * n_neighbors)
    return continuity

# Example usage:
continuity_score = calculate_continuity(x_train, x_new_train)
print(f"Continuity: {continuity_score}")


Continuity: 0.51


In [91]:
def calculate_stress(X_high, X_low):
    D_high = pairwise_distances(X_high)
    D_low = pairwise_distances(X_low)

    # Compute the stress as sum of squared differences of distances
    stress = np.sum((D_high - D_low)**2) / np.sum(D_high**2)
    return stress

# Example usage:
stress_score = calculate_stress(x_train, x_new_train)
print(f"Stress: {stress_score}")


Stress: 0.8691021885828404
